In [1]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from tqdm import tqdm

In [2]:
#load the prepared data
import pickle

with open("D:/mlds_data/text_mining/projet/prepared_data/cleaned_data_V1.txt", "rb") as fp:   # Unpickling

    DATA = pickle.load(fp)
    
    #take the 100 last examples for test
    TEST_DATA=DATA[len(DATA)-100:len(DATA)]


In [3]:
#load the saved model
output_dir="D:/mlds_data/text_mining/projet/model"
output_dir = Path(output_dir)

print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)


Loading from D:\mlds_data\text_mining\projet\model


In [16]:
# Random select of an test example and visualisation of entities
test_unit=random.choice(TEST_DATA)

#print the existing entities (real Y)
Y_entities =[]
for i in test_unit[1]['entities']:
    print(i[0],'->', i[1], i[2],':',test_unit[0][i[0]:i[1]])
    Y_entities.append(({'start':i[0],
     'end':i[1],
     'label':i[2],
     'name':test_unit[0][i[0]:i[1]]}))
    
#visualize the detected entities in this example 
from spacy import displacy
doc2 = nlp_test(test_unit[0])
displacy.render(doc2, style='ent', jupyter=True)
    

8328 -> 8333 GENE : BRCA1
13097 -> 13102 GENE : BRCA1
2044 -> 2051 GENE : CACNA1C
2146 -> 2153 GENE : CACNA1C
2306 -> 2313 GENE : CACNA1C
2384 -> 2391 GENE : CACNA1C
2457 -> 2464 GENE : CACNA1C
2766 -> 2773 GENE : CACNA1C
4813 -> 4815 GENE : GC
1256 -> 1263 GENE : CYP2C19
8371 -> 8377 DISEASE : Cancer
1668 -> 1681 DISEASE : Schizophrenia


In [178]:
#Get the pre calculated entities, and see if they were detected by the Spacy model.
import pandas as pd
import numpy as np

RES_TEST=[]
id_doc=0
for test_unit in TEST_DATA:    
    id_doc+=1
    doc2 = nlp_test(test_unit[0])
    #create a dataframe of the existing entities 
    for i in test_unit[1]['entities']:
        tmp = [id_doc,
               i[0],
               i[1],
               i[2],
               test_unit[0][i[0]:i[1]],
               "NOT_DETECTED"]#put by default the label "NOT_DETECTED", if they was detected wil update the field
        #test for each existing entities if it was detected by the model
        for ent in doc2.ents:
            if (tmp[1]==ent.start_char and tmp[2]==ent.end_char and tmp[3]==ent.label_ ):
                tmp[5]=ent.label_ #update the "NOT_DETECTED" label by the one detected
                        
        RES_TEST.append(tmp)
                    
RES_TEST = pd.DataFrame(RES_TEST,columns=['id_doc', 'start', 'end', 'label', 'name', 'Y_hat_label'])

In [179]:
import pandas as pd
import numpy as np
table1 = pd.crosstab(RES_TEST['label'],RES_TEST['Y_hat_label'], rownames=['Y'], colnames=['Y_hat'])
tmp =  table1['DISEASE'] + table1['GENE'] 
table1['rate'] = tmp / (tmp+table1['NOT_DETECTED'])
table1

Y_hat,DISEASE,GENE,NOT_DETECTED,rate
Y,,,,
DISEASE,88,0,17,0.838095
GENE,0,328,953,0.256050


In [170]:
table2 = pd.crosstab(RES_TEST['name'],RES_TEST['Y_hat_label'], rownames=['NAME'], colnames=['Y_hat'])

tmp =  table2['DISEASE'] + table2['GENE'] 
table2['rate'] = tmp / (tmp+table2['NOT_DETECTED'])
table2

Y_hat,DISEASE,GENE,NOT_DETECTED,rate
NAME,,,,
ABCC8,0,1,0,1.000000
ACACA,0,5,0,1.000000
ACE,0,0,5,0.000000
AGT,0,3,20,0.130435
AGTR1,0,1,0,1.000000
AIF1,0,3,0,1.000000
AKT3,0,3,0,1.000000
ANK2,0,0,6,0.000000
APOB,0,0,4,0.000000
